In [1]:
import pandas as pd
import numpy as np
import random


In [44]:
todo

674

In [33]:
df = pd.read_csv("data/d2_trec.csv", sep=",")

In [5]:
df.Task.unique()

array(['10', '100', '102', ..., '9994673_2', '9994673_3', '9994673_5'],
      dtype=object)

In [8]:
df1 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-01.txt.gz", error_bad_lines=False, sep="\t")
df2 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-02.txt.gz", error_bad_lines=False, sep="\t")
df3 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-03.txt.gz", error_bad_lines=False, sep="\t")
df4 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-04.txt.gz", error_bad_lines=False, sep="\t")
df5 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-05.txt.gz", error_bad_lines=False, sep="\t")
df6 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-06.txt.gz", error_bad_lines=False, sep="\t")
df7 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-07.txt.gz", error_bad_lines=False, sep="\t")
df8 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-08.txt.gz", error_bad_lines=False, sep="\t")
df9 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-09.txt.gz", error_bad_lines=False, sep="\t")
df10 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-10.txt.gz", error_bad_lines=False, sep="\t")

In [109]:
aol = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

In [25]:
# Preprocessing
aol = aol.drop_duplicates(['AnonID', 'Query'], keep='first')
aol = aol[~aol['Query'].str.lower().str.contains('|'.join(["www.", ".com", ".uk", ".us", ".edu", "google", "wiki"]), na=False)]
aol["QueryTime"] = pd.to_datetime(aol["QueryTime"])
aol = aol.dropna(subset=['Query'])
# following Personalizing Query Auto-completion for Multi-session Tasks, https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8542201
aol = aol.groupby("AnonID").filter(lambda x : len(x) >= 50)
freq_queries = aol.groupby("Query").size().sort_values(ascending=False)
aol = aol[~aol.Query.isin(freq_queries[:100].keys().tolist())]
no_nan_aol = aol[~aol["ClickURL"].isna()]

In [114]:
aol = no_nan_aol[no_nan_aol['Query'].str.split().apply(len) > 2]

In [116]:
howto = aol[aol['Query'].str.lower().str.contains("how to", na=False)]

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NUM_WORDS = 100000

# corpus = howto.Query.tolist() + df.Query.tolist()
corpus = df.Query.tolist()

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Using TensorFlow backend.


Found 19466 unique tokens.


In [6]:
path = "/Users/jarana/workspace/WikiHow-Task-Based/"

In [51]:
wiki = pd.read_csv("data/wikihowSep.csv", nrows=100)


In [78]:
wiki = pd.read_csv("data/wikihowSep.csv")
wiki = wiki.dropna()
wiki['headline'] = [i.strip() for i in wiki['headline']]

remove_digits = str.maketrans('', '', digits)
wiki['title'] = [i.translate(remove_digits) for i in wiki['title']]
for col in wiki:
    wiki[col] = [re.sub(r'[\W_]+', ' ', i).lower() for i in wiki[col]]

singleWiki = wiki[wiki['sectionLabel'] == "steps"]

In [96]:
wikiCat = pd.read_csv("data/cate.csv", sep=",", error_bad_lines=False, names=["title", "category"])

In [97]:
wikiCat['title'] = wikiCat['title'].str.replace("https://www.wikihow.com/", "").str.replace("%22","").str.replace("-", " ").str.lower()

In [98]:
wikiCat['title'] = ["how to " + i for i in wikiCat['title'].tolist()]

In [99]:
wikiCat['category'] = wikiCat['category'].str.lower()

In [102]:
travel = wikiCat[wikiCat.category.str.contains("travel")]

In [113]:
travel.title

373                        how to act on a white house tour
418                       how to adapt to brazilian culture
529                  how to add photos to an airbnb listing
724                    how to address a letter in australia
820                how to adopt a traveling lifestyle broke
1198                  how to apply pan number for a company
1267      how to apply for police clearance certificate ...
1273                how to apply for a chinese tourist visa
1282                       how to apply for a japanese visa
1290                       how to apply for a pakistan visa
1291                  how to apply for a passport for a kid
1294                    how to apply for a philippines visa
1296                       how to apply for a schengen visa
1298                      how to apply for a singapore visa
1299              how to apply for a student visa to canada
1301               how to apply for a u.s. visa from canada
1302                         how to appl

In [109]:
wiki[wiki['title'].isin(travel.title.tolist())]

,overview,headline,text,sectionLabel,title
11567,pet therapy involves interacting with a train...,ask a doctor or therapist about pet therapy,a professional referral might help you to fin...,finding a pet therapy program,how to reduce homesickness with pet therapy
11609,pet therapy involves interacting with a train...,look for therapy dog associations in your area,there are hundreds of associations that regis...,finding a pet therapy program,how to reduce homesickness with pet therapy
11610,pet therapy involves interacting with a train...,check for pet therapy programs at your univers...,universities often provide pet therapy for st...,finding a pet therapy program,how to reduce homesickness with pet therapy
11611,pet therapy involves interacting with a train...,look for other ways to interact with animals i...,while pet therapy is becoming increasingly po...,finding a pet therapy program,how to reduce homesickness with pet therapy
11612,pet therapy involves interacting with a train...,schedule regular sessions,as with any type of therapy pet therapy works...,engaging in pet therapy,how to reduce homesickness with pet therapy
11613,pet therapy involves interacting with a train...,talk to the therapist about interacting with t...,pet therapy is not just playing with a dog or...,engaging in pet therapy,how to reduce homesickness with pet therapy
11614,pet therapy involves interacting with a train...,encourage friends and family members to join you,family members and friends often report they ...,engaging in pet therapy,how to reduce homesickness with pet therapy
11615,pet therapy involves interacting with a train...,see other people,make an effort to go out isolating yourself w...,continuing to address homesickness,how to reduce homesickness with pet therapy
11668,pet therapy involves interacting with a train...,learn to embrace alone time,when you first move to a new area you may not...,continuing to address homesickness,how to reduce homesickness with pet therapy
11669,pet therapy involves interacting with a train...,appreciate a new environment,being in a new place can cause homesickness i...,continuing to address homesickness,how to reduce homesickness with pet therapy


In [111]:
travel

,title,category
373,how to act on a white house tour,"""home > categories > travel > destinations > ..."
418,how to adapt to brazilian culture,"""home > categories > travel > living overseas..."
529,how to add photos to an airbnb listing,"""home > categories > travel > lodging > """
724,how to address a letter in australia,"""home > categories > travel > living overseas..."
820,how to adopt a traveling lifestyle broke,"""home > categories > travel > """
1198,how to apply pan number for a company,"""home > categories > travel > living overseas..."
1267,how to apply for police clearance certificate ...,"""home > categories > travel > """
1273,how to apply for a chinese tourist visa,"""home > categories > travel > """
1282,how to apply for a japanese visa,"""home > categories > travel > """
1290,how to apply for a pakistan visa,"""home > categories > travel > """


In [7]:
from utils import *

max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)


Indexing word vectors.


KeyboardInterrupt: 

In [126]:
data = pd.read_csv("data/query_task_prediction.csv", sep="\t", names=["query", "task", "label"])

In [135]:
MAX_SEQUENCE_LENGTH = max([len(i.split()) for i in corpus])
x_query = pad_sequences(tokenizer.texts_to_sequences(data["query"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)

In [136]:
x_task = pad_sequences(tokenizer.texts_to_sequences(data["task"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)

In [152]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalMaxPooling1D, Dot, LSTM


q_model = Sequential([embedding_layer, GlobalMaxPooling1D()])
t_model = Sequential([embedding_layer, GlobalMaxPooling1D()])

lstm = LSTM(100)
q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
t_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
q_emb = lstm(embedding_layer(q_inp))
t_emb = lstm(embedding_layer(t_inp))
pred = Dot(-1, normalize=True)([q_emb, t_emb])
model = Model([q_inp, t_inp], pred)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit([x_query, x_task], y, batch_size=128, validation_split=0.3)

In [8]:
data = pd.read_csv(path+"data/step_task_prediction.csv", nrows=100, sep="\t", names=["step", "step_desc", "task", "label"])

In [324]:
corpus = list(set(data.step.tolist() + data.step_desc.tolist() + data.task.tolist()))


In [28]:
data = pd.read_csv(path+"data/step_prediction.csv", nrows=10000, sep="\t", names=["s1", "s2", "s1_text", "s2_text", "task"])


In [325]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)


In [281]:
multiWiki = wiki[wiki['sectionLabel'] != "steps"]

In [123]:
aol_howto = howto.Query.tolist()
gg_query = df[df.Source != "wikihow"].Query.tolist()
task = df[df.Source != "wikihow"]['Task'].tolist()
task2wiki = df[df.Source == "wikihow"].groupby("Task")['Query'].apply(list).to_dict()

x_howto = pad_sequences(tokenizer.texts_to_sequences(aol_howto), maxlen=MAX_SEQUENCE_LENGTH)
x_gg = pad_sequences(tokenizer.texts_to_sequences(gg_query), maxlen=MAX_SEQUENCE_LENGTH)

In [124]:
x_gg = model.predict(x_gg)

In [125]:
x_aol = model.predict(x_howto)

In [109]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [152]:
# test
# a = [[1,2,3,4], [5,6,7,8],[1,2,3,4], [1,24,4,1]]
# b = [[1,2,3,4], [1,24,4,1]]
# cosine_similarity(a,b)

In [156]:
sim_idx = np.argmax(cosine_similarity(x_aol, x_gg), axis=-1)

In [196]:
for i in range(1000):
    print(aol_howto[i] +"\t"+ gg_query[sim_idx[i]] + "\t" + str(task[sim_idx[i]]))
    for j in task2wiki[task[sim_idx[i]]]:
        print(j)
    print()

how to make tulle butterflies	how to attract butterflies to yourself	1655
how to grow your own flowers and sell them
how to attract butterflies

how to paint concrete floor	how to lay carpet on concrete garage floor	5765
how to install carpet on concrete (basement)
how to protect a basement from flooding

pageant hair extentions how tos	how to put in clip in hair extensions yourself	8770
how to attach clip in hair extensions

how to uninstall a kitchen facuit	how to completely uninstall a program	431
how to remove programs (windows 7)
how to access and use the bing desktop extension in windows 7

how to uninstall a kitchen faucet	how to deep clean a kitchen	10044
how to clean a kitchen
how to clean a sponge

how to change timing belts	how to change an alternator belt	3033
how to change an alternator
how to hook up jumper cables
how to loosen lug nuts

how to change hyndia timing belts	how to change an alternator belt	3033
how to change an alternator
how to hook up jumper cables
how to 

how to deal with impossible people

how to deal with adult adhd	how to deal with adhd teenager	4832
how to treat adhd in teens
how to deal with a boyfriend that has adhd
how to keep a job with adhd
how to help children with adhd

how to find royal carribean employees	what to pack for a royal caribbean cruise	1036
how to cruise
how to pack for a cruise

how to tell if your man is cheating	how to tell if your girlfriend is cheating quiz	3271
how to know if your girlfriend/boyfriend is cheating on you

how to chrome plate	how to prevent rust on chrome	410
how to remove rust from chrome
how to keep car doors from freezing shut

how to make a resume	how to write a simple resume	2172
how to write a cv (curriculum vitae)
how to make a resume
how to make your cv sound more impressive

how to make a loop bow	how to make a bow tie easy	750
how to make a bow tie
how to tie a tie
how to tie a bow tie
how to make a duct tape bow tie

how to word a wedding program	how to perform a wedding ceremony w

In [198]:
howto["QueryTime"] = pd.to_datetime(howto["QueryTime"])

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [208]:
%%time
from queue import Queue, Empty

window_size = 13
# jobs = Queue()
jobs = []
count = 0
ids = []
for index, row in howto[~howto.AnonID.isin(ids)].iterrows():
    start_time = row['QueryTime'] - pd.Timedelta(minutes=window_size)
    stop_time = row['QueryTime'] + pd.Timedelta(minutes=window_size)
#     jobs.append([row['AnonID'], row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S'), start_time, stop_time])
#     jobs.put(index)
#     jobs.append(index)
    tmp = aol[aol.AnonID == row['AnonID']]
    sessions = tmp[tmp['QueryTime'].between(start_time, stop_time)]['Query'].tolist()
    if len(sessions) > 5:
        print('\t'.join(sessions))
        print()
#     ses = '\t'.join(tmp[tmp['QueryTime'].between(start_time, stop_time)]['Query'].tolist())
#     print(ses)
#     break
#     print('data/tmp/%d_%s.csv' % (row['AnonID'], row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S')))
#     with open('data/all_ehow/%d_%s.csv' % (int(row['AnonID']), row['QueryTime'].strftime('%Y-%m-%d_%H:%M:%S')), 'w') as the_file:
#         the_file.write('%s\n' % ses)
#     break
#     print(ses)
#     with open('data/session.csv', 'a+') as the_file:
#         the_file.write('%s\n' % ses)


esmeril	emeril	quajada	how to make quajada	recipes from honduras	recipes from honduran chees	recipes from honduran cheese	nicaragua	nicaragua cheese

why people should invest money	social security	social security debate	problems with social security	why invest	how to invest	pittbull's in america	young america invest	investment info	why exercise

low dose estrogen birth control pills	how to diagnose facet hyperextension injury	have your own website	website about facet joint injuries	fairy statues	faery clocks

phi air medical	phi air medical of maryland	phi air medical of baltimore maryland	petroleum helicopters incorporated baltimore maryland	how to put on an elementary school play	elementary school play ideas	school play ideas	school play scripts

butler co ohio gov	how to get a venders licens	how to get a venders license	how to get a venders licens online form	how to get a venders license online form	butler co ohio how to get a venders license online form

butler co ohio gov	how to g

what does wear and tear on felt fabric look like	what does wear felt fabric look like	what does fiber wear felt fabric look like	what does felt fabric look like	how to fix felt fabric when worned out	how to fix felt fabric when turned out	how to fix felt fabric when lifted	ruined felt fabric	felt material launder care	felt fabric launder care	felt fabric pilling

what does wear and tear on felt fabric look like	what does wear felt fabric look like	what does fiber wear felt fabric look like	what does felt fabric look like	how to fix felt fabric when worned out	how to fix felt fabric when turned out	how to fix felt fabric when lifted	ruined felt fabric	felt material launder care	felt fabric launder care	felt fabric pilling

what does wear and tear on felt fabric look like	what does wear felt fabric look like	what does fiber wear felt fabric look like	what does felt fabric look like	how to fix felt fabric when worned out	how to fix felt fabric when turned out	how to fix felt fabric when l

KeyboardInterrupt: 

In [223]:
for idx, grp in howto.groupby("AnonID"):
    if len(grp) > 10:
        print(grp['Query'].tolist())
        print()

['how to install a glue down floor', 'skillet cookies how to make', 'how to make scales', 'how to make measurment scales', 'how to make small weight scales', 'how to catch perch', 'how to catch perch in traps', 'how to make a crawdad trap', 'how to weave an indian basket', 'how to get your drivers liscence back if convicted', 'how to get your drivers licence back if convicted', 'how to play slots']

['how to torment someone', 'how to humiliate someone', 'how to make someone misreable', 'how to drive someone crazy', 'how to get revenge on an old lover', 'advice on how to get revenge on an old lover', 'how to make an old lover suffer', 'how to say goodbye hurtfully', 'how to report child neglect in the state of new hampshire', 'how to get back on an ex lover', 'how to move on from a broken heart', 'how to send junk mail to someone else', "how to ruin someone's credit", 'how to permantlly delete information from your hard drrive', 'how to permanently delete information from your hard driv

['how to tell that you are emo', 'how to make a layout', 'how to make a banner', 'how to place an image on top', 'how to make icons', 'how to make lj icons', 'how to make layout', 'how to make layouts', 'how to make midis', 'how to add glitter to pictures', 'how to be popular', 'how to orgasm while masturbating', 'how to play tetris', 'how to write lyrics', 'how to add playlist to xanga', 'how to burn cds']

['how to make stretch mark disappear', 'how to make your face into a cartoon', 'how to turn a picture into a cartoon', 'scripts on how to make spaces', 'how to trick out pictures', 'how to pronounce lebron', 'how to change ml to l', 'how to convert milliliters to liters', 'how to convert millimeters to meters', 'how to talk to girls', 'how to curve stitching', 'how to curve stitch', 'how to curve stitch a circle', 'tutorals on how to curve stitch', 'tutorials on how to curve stitch', 'how to laminate a t-shirt', 'how to laminate an image to a t-shirt', 'how to laminate an image to 

['how to get jewlery melted down into something else', 'how to guide o be a great lesbian', 'how to guide to be a great lesbian', 'how to have lesbian sex', 'how to guide to fishing', 'how to catch bass', 'how to catch whiting fish', 'how to catch porgey fish', 'how to catch porgy fish', 'how to catch porgy fish in new york city', 'how to throw up my food so i wont gain weight', 'how to get an address for free']

['how to check inlisted assigments army', 'how to trim thieghs and butt', 'how to trim thighs and butt', 'how to obtain a death certificate tx', 'how to locate a tx inmate', 'how to purchase dallas newspapers', 'how to do a criminal check in tx', 'info on how to search public records tx', 'how to calculate due date', 'how to change father on birth certificate tx', 'how to change a childs legal last name', "how to change a child's legal last name", "how to change a child's legal last name in tx"]

['porqua how to tale ideas', 'porqua how to tile ideas', 'how to tale ideas', 'ho

['how to edit movies with nero 6', 'how to get a marrige licence in holyoke ma', 'how to get a marriage licence in holyoke ma', 'how to get a young girl to have sex', 'how to get medicare', 'how to qualify for medicare', 'how to fix a screen', 'how to fix a window screen', 'easy steps on how to true a bike wheel', 'easy steps on how to tie a bike wheel', 'how to add a video to myspace profile', 'how to make a home porn movie', 'how to fix a toothache', 'how to stop a toothache', 'how to build a laptop', 'how to calm a toothache', 'how to stop a bad toothache', 'how to get a good sound out of a amp', 'how to get a good sound out of a amp ibanez toneblaster', 'how to apply for medicare', 'how to get a robthomas something to be backstage pass', 'how to add stuff to bigornot background', 'how to hook up the motherboard model number mp4m-23agm', 'how to install a motherboard']

['guide how to program coolsat5000 to coolsat5000', 'instructions how to program satbox to satbox', 'guide how to 

['how to cook crack', 'how to make your dick bigger', 'how to make a s1 full auto bb gun', 'how to grow weed out side', 'how to guide for growin weed', 'how to guide for growing weed', 'how to draw praying hands in handcuffs', 'how to draw praying hands', 'how to make a crab pot', 'how to make a crab trap', 'how to catch hard shell crabs']

['how to make scrambled eggs', 'how to take professional photos with your digital camera', 'how to change default video', 'how to change default video playe', 'how to change default video player', 'how to change default media player', 'how to change default media player aol', 'how to make words flash in image ready', 'how to clear your cache', 'how to save imagemap', 'how to save image map', 'how to image map in imageready']

["how to simplify life 40's and 50's", 'how to find secure feeling of life long ago', 'how to find lost feeling for life', 'loosing interest in life how to get back', 'how to root japanese maple tree', 'how to root japanese map

...

['how to rock shoes without tite laces', 'how to wear shoes without laces', 'how to put on a softball uniform', 'how to wear a softball uniform', 'how to give a lap dance', 'how to kiss', 'how to hava a middle school party with botg genders attending', 'how to hava a middle school party with botg genders', 'how to have a middle school party with both genders', 'how to have a party with both genders', 'how to shave your eyebrow', 'how to win a pagent', 'how to win a pageant', 'how to eat yourself out', 'how to give head', 'how to fuck someone good', 'how to have wonderful sex', 'how to shave your pubes', 'how to find out who my friend likes', 'how to dance like a stripper', 'how to dance like a hoe', 'how to booty pop', 'how to pop your butt', 'how to dance like a black girl', 'how to make the basketball team', 'how to play basketball', 'how to make the girls track team', 'how to wax your eyebrows at home']

['how to start your own daycare center', 'how to start you own daycare center',

['song lyrics ima show you how to get your shine on', 'how to center your myspace profile', 'how to center your entire myspace profile', 'how to make your myspace profile totally different', 'how to center your entire myspace profile with all the info going down the middle', 'how to make profile snappers', 'how to make profile snappers for myspace', 'myspace profile editor how to hide bottom links and stretch tables', 'how to hide the links at the bottom of your myspace profile', 'how to make the left tables in your myspace profile longer that your right tables', 'how to make a myspace profile redirector', 'how to make a myspace profile redirection', 'how to make your left side colums longer than your right ones on myspace']

['how to be a good girlfriend', 'how to hang curtens the dighner way', 'how to hang curtains the dighner way', 'how to hang curtains the dizighner way', 'how to hang curtains', 'how to instructions to make dangly earrings free', 'how to make dangally earings', 'ho

['how to crochet flowers', 'how to make a baby sling', 'how to crochet a baby sling', 'how to tie a sling', 'how to tie a baby sling', 'how to make your own toy cars', 'how to make paper toy cars', 'how to make a baby pouch', 'how to make my own pouch', 'how to make my own baby pouch', 'how to make your own table labels for a wedding', 'how to create a website for free']

['learn how to speak italian', 'how to unscrabblem words', 'how to unscramble words', 'in myspace how to repeat your back', 'learn italian and how to speak it free', 'how to put on eye make-up', 'how to apply eye make-up', 'how to apply eye make-up with pictures', 'how to get a big butt', 'how to make a cute neopet layout with paimnt', 'how to make a cute neopet layout with paint', 'how to make a neopet layout with paint', 'how to make a guild image neopets', 'how to make a great guild neopets', 'how to animate with gimp']

['how to write formulas for sequences', 'how to convert feet into square feet', 'how to round f

['how to hide last login on myspace', 'directions on how to hide last login on mysapce', 'directions on how to hide last login on myspace', 'directions how to hide last login on myspave', 'directions on how to hide last login on myspave', 'how to hide friends on myspace', 'how to get falling hearts', 'how to hide friends but not top 8 on myspace', 'how to hide friends but not top eight friends', 'how to hide friends but not top eight on mysapce', 'how to hide all list of friends but not to hide top eight on mysapce', 'how to hide friends', 'how to hide the bottom part of page on myspace the privacy questions..etc']

['how to create an installation cd', 'how to burn copies of the sims', 'how to burn copies of the sims deluxe edition', 'how to build a well cover', 'how to build a covered well', 'how to build a leanto', 'how to build a wel leanto', 'how to build a well leanto', 'how to build a coverd well', 'how to create a bootable os disc', 'how to create a bootable windows xp disc', 'h

['free how to make her orgasem', 'free how to make her orgasemguide', 'free how to make her orgasem guide', 'how to fuck my wife good and hard', 'free online guide how to fuck my wife good and hard', 'free step by step guide how to pleasure my wife', 'free step by step guide how to make her beg me o fuck her', 'free online guide how to give her a mind blowing orgasem', 'free how to sexguide', 'free how to fuck my wife guide', 'free how to give my wife a orgasam']

['how to clean a squirell', 'how to gut a morning dove', 'how to clean and cook a morning dove', 'how to gut a rabbit', 'how to load windows 98se on a laptop', 'how to get in the bios on a laptop', 'how to texture a wall', 'how to repair a craftsman lawn tractor', 'how to fix a disk i o error in a laptop', 'how to get leads in my area', 'how to get contractor leads in my area', 'how to fix a pcmica slot', 'how to fix a pcmcia slot', 'how to fix a pcmcia slot on a satellite 405cs']

['how to properly paint and choise the right

In [17]:
%%time
todo = ["visa"]
for i in todo:
    howto = aol[aol['Query'].str.lower().str.contains(i, na=False)]
    if len(howto)> 1:
#         print(howto.AnonID.tolist())
        print()
        for i in range(len(howto)):
            row = howto.iloc[i]
            window_size = 26

#             start_time = row['QueryTime'] - pd.Timedelta(minutes=window_size)
            start_time = row['QueryTime'] 
            stop_time = row['QueryTime'] + pd.Timedelta(minutes=window_size)
            tmp = aol[aol.AnonID == row['AnonID']]
            sessions = tmp[tmp['QueryTime'].between(start_time, stop_time)]['Query'].tolist()
            print(sessions)



['visa', 'capitol one credit cards for buisness', 'small buisness grants']
['apply for visa card']
['visas']
['france visa']
['visaloo']
['chase visa', 'chase payment center']
['invisable fence']
['mastercard visa stickers']
['visa gift card', 'how paypal works']
['solicitacion de visa para eu']
['invisalign']
['citroen visa', 'citroen cx']
['visa platinum', 'creditone bank', 'unity mutual insurance']
['capital one visa']
['aol visa']
['invisalign space front']
['aaa visa', 'sears mastercard']
['example of personal letter for someone wanting work visa', 'example of personal letter for integrity']
['providianvisa', 'providian visa']
['providian visa']
['visa gift cards', 'master card gift cards', 'bicycle stores near boyston ma', 'boyston ma', 'boylston ma']
['cost of invisalign']
['michelle visage', 'essex county special transportation']
['capital one visacard']
['bank of american home equity line of credit visa access card']
['invisaline', 'invisalign']
['invisalign']
['invisalign su

['juniper bank visa']
['infomationvisastoisraelhttp']
['credit cards from aspire visa']
['30 000 dollar visa', 'high balance visa cards', 'bank of america visa card', 'chaffe mccall law firm']
['high balance visa cards', 'bank of america visa card', 'chaffe mccall law firm']
['bank of america visa card', 'chaffe mccall law firm', 'word of life school', 'word of life school in terrytown', 'word of life school in terrytown la']
['visa application', 'citibank card', 'can someone take over your car notes', 'can someone take over your car payment', 'sun finance']
['http; travel gov visa']
['providian visa', 'capitol one', 'national city mortgage', 'beneficial']
['target visa']
['aspire visa', 'city map enterprise al']
['paypalvisa', 'paypal visa']
['paypal visa']
['brides who have their visa', 'brides already has a visa', 'search brides who already has her visa', 'brides who already has her visa', 'send her to me', 'send for a bride', 'mail order bride']
['brides already has a visa', 'searc

['optomertrists-visalia. ca']
['visa card']
['visabuxxx cp']
['visabuxxx.co']
['american visa from africa']
['chase visa card']
['visa gift cards']
['chase visa credit card', 'personalized aol settigs', 'personalized aol settings']
['visa cards']
['disney visa', 'disney visa account', 'batavia idol']
['disney visa account', 'batavia idol', 'kaitlyn baldwin']
['pay online disney visa']
['televisa mexico']
['visalia school district']
['aol visa']
['delta airlines visa card', 'delta']
['us air visa card']
['usaa reloadable visa gift card', 'visa reloadable gift card for kids under 18']
['visa reloadable gift card for kids under 18']
['visa', 'low apr']
['majestic visa', 'madison square garden']
['visa bucks']
['mexico visa']
['payaspirevisacard']
['immigrant visa']
['capital one visa']
['visalia ca', 'el presidente', 'el presidente visalia', 'el presidente visalia club', 'the prez visalia club', 'el presidente club', 'central valley clubs', 'el presidente night club', 'el presidente night

['i400 visa application']
['invisalign']
['disneyvisa']
['visa', 'mastercard']
['instant approval visa credit card', 'instant approval visa credit card for people with no credit', 'no credit history instant approval visa credit card', 'no credit history instant approval visa card']
['instant approval visa credit card for people with no credit', 'no credit history instant approval visa credit card', 'no credit history instant approval visa card']
['no credit history instant approval visa credit card', 'no credit history instant approval visa card']
['no credit history instant approval visa card']
['visa credit cards for people with no credit']
['secu raleigh nc 27611 - 7503 i have a visa debit card what do i do now', 'secu raleigh nc 27611 - 7503', 'secu']
['visa debit card', 'secu visa debit card']
['secu visa debit card']
['secu visa debit card raleighl nc 274611 - 7503g.account access']
['unsecured visa application']
['its easy passport & visas longisland']
['passports and visas in g

['wachovia visa', 'visa charge cards', 'visa charge wachovia']
['visa charge cards', 'visa charge wachovia']
['visa charge wachovia']
['visaliaeric']
['visa log in']
['map of visalia ca', 'streets in visalia ca']
['streets in visalia ca']
['aol visa']
['british consulate for visa info']
['chase visa']
['visa']
['boletin de visas de marzo']
['visas bulletin.gov']
['visas bulletin .gov', 'visas.bulletin .gov']
['visas.bulletin .gov']
['greendotvisa', 'green dot visa', 'tawneestoneserie', 'tawneestonefree']
['green dot visa', 'tawneestoneserie', 'tawneestonefree']
['visa credit card']
['capitol one - visa']
['capitol one - visa no hassel miles']
['visa']
['invisable dog fence large dog collar', 'petsafe invisible dog fence large dog collar']
['majestic visa']
['aspire visa', '-']
['fiance visa application passport pics specs', 'i-129f picture size']
['visacaed', 'visa card']
['visa card']
['visa credit cards']
['visa']
['visa for china']
['chinese to get visa for america', 'visa to china'

...

['visalia and unified and school']
['chase visa']
['visa card', 'citibank', 'wal-mart', 'wal-mart credit card']
['visa rewards website']
['visa']
['listas de citas de visas en republica dominicana']
['invisable fencing']
['visa buxx card']
['visalaw']
['subject to deportation with an expired e-2 visa']
['travel.state.gov visa fri bulletin.html']
['instant visa', 'sears', 'sears portrait']
['visalia ca real estate', 'yosemite ca real estate']
['chase visa']
['visa para guatemaltecos', 'visa para trabajadores']
['visa para trabajadores']
['visalia newspaper', 'brenda wilson', 'brenda wilson ca', 'brenda wilson ca obit', 'ok lake levels']
['aaa visa', 'bny visa']
['bny visa']
['visa']
['rei visa card']
['visa']
['aol visa']
['k1visa']
['visa to go to colombia']
['visa 20gift 20car', 'american 20express 20gift 20card']
['passport& visa']
['rennaisance festival visalia', 'renaissance festival visalia']
['renaissance festival visalia']
['dan visalia times delta', 'dan visalia times delta wri

['visa account']
['zierer visa service']
['visa']
['visa']
['superior visa card']
['http visa.gov']
['japanese visa', 'japanese visa requirements', 'japanese transit visa requirements', 'hong kong visa requiremnts', 'hong kong visa', 'japan visa', 'japanes embassy manila', 'japanese embassy manila']
['japanese visa requirements', 'japanese transit visa requirements', 'hong kong visa requiremnts', 'hong kong visa', 'japan visa', 'japanes embassy manila', 'japanese embassy manila']
['japanese transit visa requirements', 'hong kong visa requiremnts', 'hong kong visa', 'japan visa', 'japanes embassy manila', 'japanese embassy manila']
['hong kong visa requiremnts', 'hong kong visa', 'japan visa', 'japanes embassy manila', 'japanese embassy manila']
['hong kong visa', 'japan visa', 'japanes embassy manila', 'japanese embassy manila']
['japan visa', 'japanes embassy manila', 'japanese embassy manila']
['visa', 'iedmanshttp', 'friedmans']
['visa rewards', 'visacheckcardrewards', 'visacheckcar

['apply for visa platinum rewards by simmons first national', 'apply for visa platinum bb&t', 'apply for platinum mastercard by franklin templeton', 'apply for visa platinum rbc centura', 'apply for visa platinum by zions', 'nc state employees credit union']
['apply for visa platinum bb&t', 'apply for platinum mastercard by franklin templeton', 'apply for visa platinum rbc centura', 'apply for visa platinum by zions', 'nc state employees credit union']
['apply for visa platinum rbc centura', 'apply for visa platinum by zions', 'nc state employees credit union']
['apply for visa platinum by zions', 'nc state employees credit union']
['invisable fence']
['pet fence invisable fence']
['invisalign method']
['invisalign', 'metropolitan', 'metropolitan dental']
['capital one visa card']
['costa rican visa', 'costa rican american visa', 'costa rican visa to visit merican visa', 'costa rican visa to visit american']
['costa rican american visa', 'costa rican visa to visit merican visa', 'costa

['instant student visa applications']
['invisa fast', 'imvisa fast', 'hidden deck fasteners', 'hidden deck fasteners without cutting']
['imvisa fast', 'hidden deck fasteners', 'hidden deck fasteners without cutting']
['visa']
['resident visas for england']
['how to get a residence visas from england']
['disney visa card from chase']
['visa gift card']
['visa from mexico']
['visa buxx']
['change visa number for billiing', 'change visa number for billing']
['change visa number for billing']
['all estate for sale visalia calif']
['allestate real estate visalia calif', 'allstate real estate visalia calif', 'real estate for sak']
['allstate real estate visalia calif', 'real estate for sak']
['visalia christian school visalia ca']
['visa expo', 'humournet']
['targetvisa', 'target visa']
['target visa']
['invisablechildren']
['k19 visa', 'law for marry foriegn woman', 'us. law for marry foriegn woman', 'k c visa', 'k3 visa', 'k3 visa rules']
['k c visa', 'k3 visa', 'k3 visa rules']
['k3 visa'

['h1b visa']
['aolvisa', 'aol visa']
['aol visa']
['france visa']
['lowesvisa platinum card']
['aspire visa']
['alo visa reward points', 'aol online reward points', 'aol visa reward points', 'aol visa card']
['aol visa reward points', 'aol visa card', 'redeem aol visa points', 'aol visa points', 'aol visa rewards']
['aol visa card', 'redeem aol visa points', 'aol visa points', 'aol visa rewards']
['redeem aol visa points', 'aol visa points', 'aol visa rewards']
['aol visa points', 'aol visa rewards']
['aol visa rewards']
['macys visa', 'visa']
['visa']
['visagedayspa.co']
['visa']
['instant approval visa']
['aspire visa']
['visa', '-', 'capital one', 'capital one payments', 'capital one account']
['visa versa', 'napolians']
['revisame miras te miro']
['passport visa sudan']
['north park mall visa gift cards']
['visiter visa canada']
['visa requirements mexico', 'passport requirements mexico']
['city of visalia jobs']
['walmart jobs visalia california', 'walmart']
['office team visalia 

In [122]:
for i in range(len(howto)):
    row = howto.iloc[i]
    window_size = 10000

#     start_time = row['QueryTime'] - pd.Timedelta(minutes=window_size)
    start_time = row['QueryTime']
    stop_time = row['QueryTime'] + pd.Timedelta(minutes=window_size)
    tmp = aol[aol.AnonID == row['AnonID']]
    sessions = tmp[tmp['QueryTime'].between(start_time, stop_time)]['Query'].tolist()
    if len(sessions) > 5:
        print(sessions)

['how to make tulle butterflies', 'shimmering butterfly party', 'order birthday cakes online', 'virginia guardianship of a minor', 'when is it necessary to be guardian of minors estate virginia', 'purple foot baby shower', 'baby shower purple foot', 'baby shower purple']
['how to make quajada', 'recipes from honduras', 'recipes from honduran cheese', 'clerks of courts miami-dade', 'mane and tails', 'how to relief gas', 'gas relief home remedies', 'academy country awards', 'madden 2006 cheats', 'bad and bath']
['how to bleed the brakes on a case backhoe', 'case backhoe 580l recalls', 'exercises with the medicine ball', 'new xbox games', 'starting a small bible study group', 'john denver lyrics', 'christian contact tables for myspace', 'structure of the heart', 'keith anderson bodybuilding pics']
['how to make oyster sauce', 'clayton modular homes', 'dr. colleen kraft virginia', 'virginia developmental pediatrician kraft', 'james sellman psychiatrist', 'symptoms of heart attack in women'

['how to build up connective tissue in the body', 'build up connective tissue via supplements', 'supplements to loosen up myofascial tissue', 'julie hewett cosmetics', 'posterior facet stabilization', 'improvised songs by jenny lin', 'natural spinal stabilization', 'get your brother to leave his wife', 'jump off a bridge', 'rheumatoid arthritis and spondylosis', 'von berg cosmetics', 'diagnosing facet joint dysfunction', 'dangers of clomid to women', 'plastic surgery for cellulite', 'add tissue filler to your spine', 'vitamin b12 and spinal instability', 'natural treatment for endometriomas', 'hormone responsible for breast development', 'beauty by tova', 'pumice in spine for ligament laxity', 'severe ligament pain and treatment', 'delaware and donor eggs', 'klonopin and weight gain', 'nitric oxide patch', 'topical nitric oxide cream for ligaments', 'teas that strengthen ligaments', 'matte mineral makeup', 'humane way to kill lobsters and peta', 'mineral makeup without mica', 'mica-fre

['activex how to remove free', 'american staffordshire terrier cincinnati ohio', 'a letter i am hurt you forgot valentines day', 'a letter poem i am hurt mad he forgot valentines day', 'things a woman needs to feel loved', 'under 20 clubwear', 'dress-up games for a child', 'clubwear exotic dancewear', 'four-way cocktail dress', 'stripper dance dancewear', 'neon yellow tie top exotic dancer stripper private', 'slinky knit grecian catsuit with deep v-front', 'sexy baby blue fishnet 3 4 sleeve top & booty shorts', 'stripper dance dancewear sale', 'wholesale stripper dance dancewear sale', 'wholesale dance clubwear', 'tax i.d. how to get', 'how to start a clothing store', 'wholesale 1.00 clothing fashion', 'butler co ohio gov', 'wholesale dancewear clubwear exotic', 'hot pink school girl set wholesale', 'pleaser boots wholesale', 'double slit open skirt set wholesale', 'boots wholesale 10.00 each', 'wholesale dancewear clubwear exotic 3.00 each', 'lingerie wholesale manufacturer', 'is the 

['how to do my own web site', 'free web space with shopping cart', 'how to design my own web page', 'html how to', 'upload them to your internet service provider isp via ftp', 'work at home no money down', 'work at home with a free web page', 'wholesale with a free web page', 'wholesale bulk drop cloth painters canvas', 'painters drop cloth floor cover canvas wholesale rags', '50 lbs rags', 'shop rags wholesale', '50 lbs shop rag red', '50 lb shop rags red', 'auto shop cotton rags', 'auto shop cotton rags wholesale', 'cotton huck rag', 'red cotton shop rags 50 lb sale', 'plants have feeling tesla broccoli', 'can plants feel ax studie electrodes', 'experiments prove plants have feelings', 'how to keep the school from drugging my kid']
['how to design my own web page', 'html how to', 'upload them to your internet service provider isp via ftp', 'work at home no money down', 'work at home with a free web page', 'wholesale with a free web page', 'wholesale bulk drop cloth painters canvas', 

['how to cite online encyclopedias in mla', 'encyclopedia britannica online', "don't forget about breakfast time", 'song with the lyrics do i know you from somewhere by madonna', 'lyrics to skin by madonna', 'what does an aerobic activity actually do for the body', 'myspace quote icons', 'polytene salivary glands in drosophila', 'in what cells does synapsis occur', 'chromocenter in drosophila chromosomes', 'synapsis in drosophila salivary glands', 'number of chromosomes in drosophila melanogaster', 'do drosophila salivary glands have the monoploid or diploid chromosome number', 'number of chromosomes in drosophila salivary glands', 'reciprocal crosses of an autosomal gene', 'cross between and xxy female and an xy male in drosophila', 'viable and inviable drosophila offspring', 'inviable drosophila offspring', 'what does a chi square test value of 0 mean', 'p1 generation of dihybrid cross', 'myspace contact tables', 'iupac name of isoamyl acetate', 'how to make isoamyl acetate', 'synthe

...

['how to cook pork c', 'cooking pork chops', 'bake pork chops recipes', 'broil pork chops recipes', 'fried pork chops', 'gun range philadelphia pa area', 'yaesu ft 1000mp mk v', 'yaesu ft1000mp mkv', 'sample letters of character witnesses', 'sample letters of character', 'the food network']
['how to buy car battery', 'chrysler car battery', 'auto parts stores in philadelphia pa', 'movies with and island people stranded', 'firearms safety training', 'video firearms safety training', 'free gun safety training', 'minelab metal detectors']
['how to clean glass', 'the worlds strongest incense', 'wandering omen enterprises', 'aftermarket 2001 saturn sc2 parts', 'xbox 360 vs ps3', 'united states air force']
['how to lose 80 pounds', 'had a bad day lyrics', 'i heart huckabees', 'free college money for independent students with low gpa', 'low gpa scholarshi', 'what language is spoken most in the world', 'long island ice tea ingredients', 'animated screen savers', 'reverend james lewis']
['baske

['how to organize your notes', 'take this love reggae song', 'reggae song jr reed', 'take is love and recieve you is all i need i got no trick up my sleeve', 'receive reggae song', 'meger evers college', 'medgar evers college', 'be wise junior kelly reggae song', 'if love so nice reggae song', 'god gave me style god gave me grace rap song', 'god gave me style rap song']
['how to grow taller', 'inner thigh pain', 'tingling in feet when crossing legs', 'glow in the dark fabric', 'glow in the dark paper', 'how to make picture frames', 'how to care diabetes']
['how to make picture frames', 'how to care diabetes', 'right by your side jadakiss hip hop song', 'by your side jadakiss hip hop song', 'mechanical reasoning sample test', 'how big is 4x6 pics']
['how to care diabetes', 'right by your side jadakiss hip hop song', 'by your side jadakiss hip hop song', 'mechanical reasoning sample test', 'how big is 4x6 pics', 'what is the dark ages', 'what is was dark ages']
['how to be a good sales m

['how to report harassment on aol journals', 'harassment on aol', 'marito della falchi', 'pasta con i broccoli', 'storia di gino bartali', "gino bartali l'intramontabile", 'grandi scienziati anziani', 'persone sepolte a san miniato', 'san miniato al monte cimitero', 'help on defamation in chat rooms', 'defamation in a chatroom', 'defamation in aol chat rooms', 'come tagliano il marmo di carrara', 'doubletree club las vegas', 'mormon temple mesa arizona', 'lsd temple mesa arizona', 'il fiume da carrara a massa carrara', 'cave di marmo e la silicosi', 'la storia della pizza', 'la storia dei pizzaioli', 'saper fare una pizza', 'derisione degli immigranti italiani']
['how to eliminate garden pests', 'recipe for paella', 'arlington natl cemetery honor guard', 'strudel con phillo dough', 'sara moulton recipe', 'apple strudel with phyllo dough', 'italia canzoni in testa', "detti di d'annunzio", 'barzellette sul viagra', 'classifica canzoni italiane', 'rai tv in diretta', 'chi sono i beccamort

['how to keep a hard penis longer', 'sex with boobs', 'how to please a man', 'grounds for termination under 90 day probation period', 'do you have to have reasons to terminate employee under 90 day probation period', 'laws in wa to terminate employee under 90 day probation period', 'probation period at assisted living facilities']
['how to lose cellulite on buttocks quick', 'cheap liposuction seattle wa', 'taking ua how to cover vicodin in test', 'how long does vicodin stay in your system', 'victoria secrets swimsuits', 'call tunes samsung a920']
['how to make belly bands', 'levis misses mid rise bootcut jeans', 'fruit lockers game', 'miniature bernese mountain dogs', 'miniature bernese mountain dog in ohio', 'illnesses in dogs', 'alphabetical diseases in dogs', 'glucosamine for dogs']
['how to stop car breaks squeaking', 'plant by numbers', 'song simple gifts', 'spanky from our gang', 'red clover herbal', 'diet & fitness coach jorge cruise', 'contact zone agility equipment', 'three pa

KeyboardInterrupt: 